In [16]:
import numpy as np
from skimage import measure, transform
from skimage.measure import label, regionprops

from PIL import Image
import matplotlib.pyplot as plt
import glob

In [17]:
def load_image(file_path):
    image = Image.open(file_path)
    return image.convert('L')   # 그레이스케일로 변환

def find_properties(binary_particle):
    props = measure.regionprops(binary_particle.astype(int))[0]
    original_particle_area = np.sum(binary_particle)
    bbox = props.bbox
    orientation = props.orientation
    centroid = props.centroid
    return original_particle_area, bbox, orientation, centroid

def add_padding(binary_particle):
    rows, cols = binary_particle.shape
    diag = int(np.sqrt(rows**2 + cols**2))
    pad_width = ((diag - rows) // 2, (diag - cols) // 2)
    padded_particle = np.pad(binary_particle, ((pad_width[0], pad_width[0]), (pad_width[1], pad_width[1])), mode='constant', constant_values=0)
    return padded_particle, pad_width

def process_particle(padded_particle, orientation, centroid, pad_width, bbox):
    # 이미지 회전
    rotation_angle = (np.pi/2 - orientation) * (180/np.pi)
    new_centroid = (centroid[0] + pad_width[0], centroid[1] + pad_width[1])
    rotated_particle = transform.rotate(padded_particle, rotation_angle, order=0, preserve_range=True, center=new_centroid)

    # 회전된 이미지의 가로세로 비율 조정
    props_rotated = measure.regionprops(rotated_particle.astype(int))[0]
    rotated_bbox = props_rotated.bbox
    rotated_height = rotated_bbox[2] - rotated_bbox[0]
    rotated_width = rotated_bbox[3] - rotated_bbox[1]
    
    original_aspect_ratio = (bbox[3] - bbox[1]) / (bbox[2] - bbox[0])
    rotated_aspect_ratio = rotated_width / rotated_height
    
    # 가로세로 비율에 맞게 이미지 크기 조정
    if rotated_aspect_ratio > original_aspect_ratio:
        # 회전된 이미지가 너무 넓으면 높이를 늘립니다.
        new_height = rotated_width / original_aspect_ratio
        scaling_factor = new_height / rotated_height
        new_rotated_particle = transform.rescale(rotated_particle, (scaling_factor, 1), order=0, preserve_range=True)
    else:
        # 회전된 이미지가 너무 높으면 너비를 늘립니다.
        new_width = rotated_height * original_aspect_ratio
        scaling_factor = new_width / rotated_width
        new_rotated_particle = transform.rescale(rotated_particle, (1, scaling_factor), order=0, preserve_range=True)

    return new_rotated_particle

def adjust_particle_scale(rotated_particle, original_axis_lengths):
    # 회전된 이미지에서 입자의 regionprops를 계산합니다.
    props = regionprops(rotated_particle.astype(int))[0]
    orientation = props.orientation
    centroid = props.centroid

    # 주축을 따라 입자의 길이를 측정합니다.
    axis_length = 2 * props.major_axis_length / 2  # regionprops는 지름을 제공하므로 반으로 나눕니다.
    # 직교축을 따라 입자의 길이를 측정합니다.
    ortho_axis_length = 2 * props.minor_axis_length / 2  # 마찬가지로 반으로 나눕니다.

    # 원본의 축 길이와 비교하여 스케일링 인자를 계산합니다.
    scale_factor_axis = original_axis_lengths['axis'] / axis_length
    scale_factor_ortho = original_axis_lengths['ortho'] / ortho_axis_length

    # 이미지를 스케일링합니다.
    scaled_particle = transform.rescale(rotated_particle, (scale_factor_ortho, scale_factor_axis), order=0, preserve_range=True, anti_aliasing=False)
    scaled_particle = (scaled_particle > 0.5).astype(int)  # 이진화

    return scaled_particle

def save_and_show_image(image_array, file_name='final_scaled_image.png'):
    # 이미지를 저장하고 화면에 보여줍니다.
    scaled_image = (1 - image_array) * 255  # 이진 이미지를 다시 그레이스케일로 변환
    image = Image.fromarray(scaled_image.astype(np.uint8))
    image.save(file_name)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

def main():
    image_files = glob.glob('*.png')
    if not image_files:
        raise ValueError("이미지를 찾을 수 없습니다.")

    for idx, file_path in enumerate(image_files):
        particle_image = load_image(file_path)
        particle_array = np.array(particle_image)
        binary_particle = particle_array < 128

        original_particle_area, bbox, orientation, centroid = find_properties(binary_particle)
        original_width = bbox[3] - bbox[1]
        original_height = bbox[2] - bbox[0]

        padded_particle, pad_width = add_padding(binary_particle)
        final_particle = process_particle(padded_particle, orientation, centroid, pad_width, bbox)

        # 원본 입자의 주축과 직교하는 축의 길이를 계산합니다.
        original_axis_lengths = {
            'axis': 2 * props.major_axis_length / 2,
            'ortho': 2 * props.minor_axis_length / 2
        }

        # 스케일링 함수를 호출합니다.
        scaled_particle_image = adjust_particle_scale(final_particle, original_axis_lengths)

        output_file_name = f'processed_{idx}.png'
        save_and_show_image(scaled_particle_image, output_file_name)

In [18]:
if __name__ == '__main__':
    main()

NameError: name 'props' is not defined